<a href="https://colab.research.google.com/github/ContextLab/cs-for-psych/blob/master/slides/module_6/permutation_tests_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codebreaking using permutation tests

Congratulations, this is the day you've been waiting for: it's time for you to upgrade your [5K1ll5](https://www.urbandictionary.com/define.php?term=5K1ll3d) by becoming a [*1337 H4X0R*](https://www.urbandictionary.com/define.php?term=1337%20h4x0r)!

![hacking](https://i.giphy.com/media/3oEjHWbXcpeKhTktXi/giphy.webp)

At [this point in our course](https://github.com/ContextLab/cs-for-psych/tree/master/slides#module-6-interrogating-your-data), you're now well-acquainted with the fundamentals of Python, and you've gotten practice with some really [powerful libraries](https://github.com/ContextLab/cs-for-psych/tree/master/slides#module-4-external-libraries).  Now it's time to put your knowledge and skills to the test by doing some simple code-breaking (and learning about permutation tests at the same time).

## Overview

We'll be implementing a simple encryption/decryption system, and then using permutation tests to "break" each other's encrypted messages.

First let's load in some useful libraries and a character lookup table:

In [145]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

ascii_table = pd.read_csv('https://raw.githubusercontent.com/ContextLab/cs-for-psych/master/slides/module_6/ascii_table.csv', dtype=str)
ascii_table['ascii code'] = ascii_table['ascii code'].apply(int)
ascii_table.set_index('ascii code', inplace=True)
ascii_table.sort_index(inplace=True)

ascii_table.head()

,character,binary
ascii code,,
32,,00100000
33,!,00100001
45,-,00101101
46,.,00101110
65,A,01000001



## Encoding (and encrypting) messages

We'll use a simple algorithm to encrypt a message, encoded as a string of human-readable characters (all of this is provided for you in the `encode` function):
1. Turn the string of human-readable characters into a binary sequence (i.e., a sequence of 0s and 1s).  Each character is represented by one byte (a sequence of 8 bits-- i.e., 0s and 1s).
2. [Circularly shift](https://en.wikipedia.org/wiki/Circular_shift) the binary sequence by some number of positions, $k$ ($k$ will be your encryption key).

The idea of applying a circular shift is to move each element in the sequence ahead by $k$ positions.  Then, for the last $k$ elements, loop their positions around back to the front of the sequence.  Here's a schematic of what it looks like to circularly shift a sequence of 8 bits forward by 1 position:

![circular shift](https://upload.wikimedia.org/wikipedia/commons/3/37/Rotate_right.svg)

This results in a new "scrambled" binary sequence.

**_Important note_: the lookup table (for converting characters to bytes) may not include every character in the message.  If a character is missing from the lookup table, the encoder will generate a *random* set of 8 0s and 1s for that character.**


In [172]:
def circshift(s, shift):
  shift %= len(s) #think about what this line does and why it's needed...
  return s[shift:] + s[:shift]

In [173]:
def encode(msg, encrypt_key=0):
  x = ''
  for c in msg:
    try:
      x += ascii_table.query(f'character == "{c}"')['binary'].values[0]
    except: #generate a random sequence of 0s and 1s
      n = np.random.randint(8) + 1 #number of 1s
      chars = [*(['1'] * n), *(['0'] * (8 - n))]
      x += ''.join(np.random.permutation(chars).tolist())
  
  return circshift(x, encrypt_key)


## Decoding (and decrypting) messages

If you knew the encrypted string's key, you could simply:
1. Apply a circular shift of $-k$ to offset the "encryption"
2. Break the sequence into bytes
3. Use a reverse lookup table to turn each byte back into a human-readable character

I've provided the `decode` function below for convenience.  Given an encrypted message and an encryption key, the function attempts to use the character lookup table to replace each byte in the message with a human-readable character.  If no valid character is found, any illegal byte sequences are replaced with "\*".

The challenge is that, in this exercise, you *won't* always know the encryption key.


In [142]:
def decode(x, encrypt_key=0):
  msg = ''
  for c in [''.join(b) for b in np.split(np.array(list(circshift(x, -encrypt_key))), len(x) / 8)]:
    try:
      msg += ascii_table.query(f'binary == "{c}"')['character'].values[0]
    except:
      msg += '*'
  return msg


## Code breaking

Your first job is to write a function, `decrypt`, to reliably(!) decrypt a message containing the characters a--z (lowercase letters), A--Z (uppercase letters), and some basic punctuation (!, -, ., and " ").  You can assume that the original message would make sense (e.g., to a fluent English speaker).

Next, you'll use the decryption function (or snippets from it) to create a permutation test of whether a given message was encrypted with a specific key.  You should implement 

In [ ]:
def decrypt(x):
  ### BEGIN YOUR CODE
  pass
  ### END YOUR CODE
  return msg #best guess at decoded message

In [ ]:
def permutation_test(x, encrypt_key):
  ### BEGIN YOUR CODE
  pass
  ### END YOUR CODE
  return p #p-value (estimated probability that the given encrypt_key is *not* correct)

# Some examples to get you started

Let's explore a few simple examples to explore how `encode` and `decode` work.  We'll start by using an encryption key of 5 to encode and encrypt the string `'this is a test'`:

In [175]:
test_message = 'this is a test'
key = 5
encrypted_message = encode(test_message, encrypt_key=key)
print(encrypted_message)

1000110100001101001011100110010000001101001011100110010000001100001001000000111010001100101011100110111010001110


If we try to decode the encrypted message with the wrong key, we get gibberish:

In [180]:
guess = 0
print(decode(encrypted_message, encrypt_key=guess))

**.d*.d*****n*


But if we decode the encrypted message with the correct key, we get back the original message:

In [181]:
print(decode(encrypted_message, encrypt_key=key))

this is a test


We can also add in some "illegal" characters that aren't in the lookup table, to see how that can make things more...interesting:

In [182]:
decode(encode('Is this a test? Or is this just practice? Or both!?'))

'Is this a testw Or is this just practice* Or both!*'

# Now for some "real" messages...

I've encrypted a set of 83 messages using a new randomly selected encryption key for each message:

In [231]:
encrypted = pd.read_csv('https://raw.githubusercontent.com/ContextLab/cs-for-psych/master/slides/module_6/encrypted.csv').drop(['Unnamed: 0'], axis=1)
encrypted.head()

,message
0,1100001001000000110001001101111011011110110101...
1,1100101011101000111010101101110011010010110000...
2,1000010110111001100100001000000110000101101110...
3,1100110011100100000011101000110111100100000011...
4,0011010010110000100101110001000000100100001101...


Can you recover the keys and decode the messages?  Good luck!